In [ ]:
# Configure AWS profile for local development
%env AWS_PROFILE=platform-developer

In [ ]:
import sys
import os
from pathlib import Path

# Add src to sys.path
src_path = Path("..").resolve() / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from adapters.axiell import config
from adapters.utils.iceberg import (
    get_rest_api_catalog_params,
    get_local_catalog_params,
    get_table,
)
from adapters.utils.schemata import SCHEMA
from adapters.utils.window_store import WINDOW_STATUS_SCHEMA
from pyiceberg.catalog import load_catalog
from pyiceberg.exceptions import NoSuchTableError

In [ ]:
def recreate_axiell_table(target: str = "local"):
    """
    Recreate the Axiell Iceberg table.

    Args:
        target: "local" or "remote" (REST API)
    """
    if target == "local":
        print(f"Recreating LOCAL table: {config.LOCAL_NAMESPACE}.{config.LOCAL_TABLE_NAME}")
        params = get_local_catalog_params(db_name=config.LOCAL_DB_NAME)
        catalog_name = "local"
        namespace = config.LOCAL_NAMESPACE
        table_name = config.LOCAL_TABLE_NAME
    elif target == "remote":
        print(f"Recreating REMOTE table: {config.REST_API_NAMESPACE}.{config.REST_API_TABLE_NAME}")
        params = get_rest_api_catalog_params(s3_tables_bucket=config.S3_TABLES_BUCKET)
        catalog_name = "s3tablescatalog"
        namespace = config.REST_API_NAMESPACE
        table_name = config.REST_API_TABLE_NAME
    else:
        raise ValueError(f"Unknown target: {target}")

    catalog = load_catalog(catalog_name, **params)
    identifier = f"{namespace}.{table_name}"

    try:
        print(f"Dropping table {identifier}...")
        if target == "remote":
            # S3 Tables requires purge_requested=True to drop tables
            catalog.drop_table(identifier, purge_requested=True)
        else:
            # SqlCatalog (local) does not support the purge argument
            catalog.drop_table(identifier)
        print("Table dropped.")
    except NoSuchTableError:
        print("Table did not exist.")

    print(f"Creating table {identifier}...")
    # We use get_table to handle namespace creation and table creation with schema
    get_table(
        catalogue_namespace=namespace,
        table_name=table_name,
        catalogue_name=catalog_name,
        create_if_not_exists=True,
        schema=SCHEMA,
        **params
    )
    print("Table created successfully.")

In [ ]:
# Recreate local table
recreate_axiell_table(target="local")

In [ ]:
# Recreate remote table (be careful!)
# recreate_axiell_table(target="remote")

In [ ]:
def recreate_window_status_table(target: str = "local"):
    """
    Recreate the Window Status Iceberg table.

    Args:
        target: "local" or "remote" (REST API)
    """
    if target == "local":
        print(f"Recreating LOCAL window status table: {config.LOCAL_WINDOW_STATUS_NAMESPACE}.{config.LOCAL_WINDOW_STATUS_TABLE}")
        params = get_local_catalog_params(db_name=config.LOCAL_WINDOW_STATUS_DB_NAME)
        catalog_name = "local"
        namespace = config.LOCAL_WINDOW_STATUS_NAMESPACE
        table_name = config.LOCAL_WINDOW_STATUS_TABLE
    elif target == "remote":
        print(f"Recreating REMOTE window status table: {config.WINDOW_STATUS_NAMESPACE}.{config.WINDOW_STATUS_TABLE}")
        params = get_rest_api_catalog_params(s3_tables_bucket=config.S3_TABLES_BUCKET)
        catalog_name = "s3tablescatalog"
        namespace = config.WINDOW_STATUS_NAMESPACE
        table_name = config.WINDOW_STATUS_TABLE
    else:
        raise ValueError(f"Unknown target: {target}")

    catalog = load_catalog(catalog_name, **params)
    identifier = f"{namespace}.{table_name}"

    try:
        print(f"Dropping table {identifier}...")
        if target == "remote":
            # S3 Tables requires purge_requested=True to drop tables
            catalog.drop_table(identifier, purge_requested=True)
        else:
            # SqlCatalog (local) does not support the purge argument
            catalog.drop_table(identifier)
        print("Table dropped.")
    except NoSuchTableError:
        print("Table did not exist.")

    print(f"Creating table {identifier}...")
    get_table(
        catalogue_namespace=namespace,
        table_name=table_name,
        catalogue_name=catalog_name,
        create_if_not_exists=True,
        schema=WINDOW_STATUS_SCHEMA,
        # No manual partitioning (unpartitioned)
        partition_spec=None,
        **params
    )
    print("Table created successfully.")

In [ ]:
# Recreate local window status table
recreate_window_status_table(target="local")

In [ ]:
# Recreate remote window status table (be careful!)
# recreate_window_status_table(target="remote")